In [1]:
#OPTIONAL: uncomment to install packages if necessary

# %pip install pandas
# %pip install scikit-learn
# %pip install ta

Preprocessing the dataset

In [2]:
import pandas as pd

df_raw = pd.read_csv("crypto.csv")

df_raw.head()

,OpenDt,CRVUSDT-open,CRVUSDT-high,CRVUSDT-low,CRVUSDT-close,CRVUSDT-volume,ZILUSDT-open,ZILUSDT-high,ZILUSDT-low,ZILUSDT-close,...,EGLDUSDT-open,EGLDUSDT-high,EGLDUSDT-low,EGLDUSDT-close,EGLDUSDT-volume,LTCUSDT-open,LTCUSDT-high,LTCUSDT-low,LTCUSDT-close,LTCUSDT-volume
0,2020-09-04 06:00:00,2.962,2.975,2.800,2.814,233206.032,0.01617,0.01630,0.01588,0.01592,...,17.524,17.563,17.000,17.027,13078.587,48.77,48.77,48.20,48.21,23686.98714
1,2020-09-04 07:00:00,2.812,2.933,2.797,2.919,422883.332,0.01594,0.01599,0.01501,0.01591,...,17.029,17.900,16.710,17.129,24945.703,48.20,48.93,47.37,48.83,46079.69323
2,2020-09-04 08:00:00,2.919,3.237,2.915,3.171,599623.833,0.01592,0.01654,0.01588,0.01635,...,17.133,17.344,17.102,17.277,7945.829,48.78,49.92,48.75,49.67,73098.31532
3,2020-09-04 09:00:00,3.171,3.294,3.100,3.282,392916.735,0.01638,0.01689,0.01621,0.01684,...,17.277,17.277,16.918,17.121,15481.387,49.68,51.54,49.34,51.45,73369.67310
4,2020-09-04 10:00:00,3.272,3.278,3.150,3.157,374373.881,0.01683,0.01717,0.01672,0.01703,...,17.133,17.750,17.100,17.342,19422.335,51.42,52.38,51.00,51.42,81565.19476


In [3]:
# do not run this more than once without resetting kernel
def rename_cols(name):
    divider = name.index('-')
    return name[divider+1:] + "-" + name[:divider-4]

rename_map = {col: rename_cols(col) for col in df_raw.columns[1:]}

rename_map["OpenDt"] = "datetime"

df_raw.rename(columns=rename_map, inplace=True)

df_raw["datetime"] = pd.to_datetime(df_raw["datetime"])

df_raw.head()

,datetime,open-CRV,high-CRV,low-CRV,close-CRV,volume-CRV,open-ZIL,high-ZIL,low-ZIL,close-ZIL,...,open-EGLD,high-EGLD,low-EGLD,close-EGLD,volume-EGLD,open-LTC,high-LTC,low-LTC,close-LTC,volume-LTC
0,2020-09-04 06:00:00,2.962,2.975,2.800,2.814,233206.032,0.01617,0.01630,0.01588,0.01592,...,17.524,17.563,17.000,17.027,13078.587,48.77,48.77,48.20,48.21,23686.98714
1,2020-09-04 07:00:00,2.812,2.933,2.797,2.919,422883.332,0.01594,0.01599,0.01501,0.01591,...,17.029,17.900,16.710,17.129,24945.703,48.20,48.93,47.37,48.83,46079.69323
2,2020-09-04 08:00:00,2.919,3.237,2.915,3.171,599623.833,0.01592,0.01654,0.01588,0.01635,...,17.133,17.344,17.102,17.277,7945.829,48.78,49.92,48.75,49.67,73098.31532
3,2020-09-04 09:00:00,3.171,3.294,3.100,3.282,392916.735,0.01638,0.01689,0.01621,0.01684,...,17.277,17.277,16.918,17.121,15481.387,49.68,51.54,49.34,51.45,73369.67310
4,2020-09-04 10:00:00,3.272,3.278,3.150,3.157,374373.881,0.01683,0.01717,0.01672,0.01703,...,17.133,17.750,17.100,17.342,19422.335,51.42,52.38,51.00,51.42,81565.19476


In [4]:
df_raw.count()

df_raw.isnull().values.any()

np.False_

In [5]:
df = (
    pd.wide_to_long(
        df_raw,
        stubnames=["open", "high", "low", "close", "volume"],
        i="datetime",
        j="symbol",
        sep="-",
        suffix=".+",
    )
    .reset_index()
)

df = df.sort_values(['symbol','datetime']).set_index(['symbol','datetime'])

df

open     high      low    close      volume
symbol datetime                                                           
ADA    2020-09-04 06:00:00  0.09716  0.09766  0.09502  0.09502  22918282.0
       2020-09-04 07:00:00  0.09506  0.09712  0.09215  0.09685  66824964.9
       2020-09-04 08:00:00  0.09682  0.10012  0.09680  0.09944  44941018.0
       2020-09-04 09:00:00  0.09945  0.10237  0.09886  0.10219  30595680.1
       2020-09-04 10:00:00  0.10218  0.10392  0.10150  0.10227  44413004.8
...                             ...      ...      ...      ...         ...
ZRX    2024-02-19 22:00:00  0.36550  0.36740  0.36480  0.36640    316212.0
       2024-02-19 23:00:00  0.36680  0.36720  0.36330  0.36540    206017.0
       2024-02-20 00:00:00  0.36540  0.36780  0.36090  0.36240    543901.0
       2024-02-20 01:00:00  0.36210  0.36570  0.35730  0.36250    457941.0
       2024-02-20 02:00:00  0.36270  0.36560  0.36230  0.36490    278950.0

[1485337 rows x 5 columns]

In [6]:
# Feature Engineering
from ta.momentum import RSIIndicator, StochasticOscillator, ROCIndicator
from ta.trend import EMAIndicator, MACD, CCIIndicator
from ta.volatility import BollingerBands
import numpy as np

c, h, l, v = df['close'], df['high'], df['low'], df['volume']

# ---- Params for hourly (4h params × 4)
ROC_WINS   = [4, 12, 168]
EMA_PAIRS  = [(336, 672)]
RSI_WINS   = [24, 56, 104]
CCI_WINS   = [40, 80]
BB_WINS    = [40, 80]
STO_KS     = [24, 56]
VOL_WIN    = 80
VOL_MA_VOL = 168
VOL_ROC_W  = [168, 336]

fe = pd.DataFrame(index=df.index)

# ROC
for n in ROC_WINS:
    fe[f'roc_{n}'] = ROCIndicator(close=c, window=n).roc()

# EMA cross (diff & ratio)
for s, L in EMA_PAIRS:
    ema_s = EMAIndicator(close=c, window=s).ema_indicator()
    ema_L = EMAIndicator(close=c, window=L).ema_indicator()
    fe[f'ema_diff_{s}_{L}']  = ema_s - ema_L
    fe[f'ema_ratio_{s}_{L}'] = ema_s / ema_L - 1.0

# RSI + lag2
for n in RSI_WINS:
    r = RSIIndicator(close=c, window=n).rsi()
    fe[f'rsi_{n}'] = r
    fe[f'rsi_{n}_lag2'] = r.shift(2)

# MACD histogram + lag2 (12,26,9)
macd_hist = MACD(close=c, window_fast=12, window_slow=26, window_sign=9).macd_diff()
fe['macd_hist'] = macd_hist
fe['macd_hist_lag2'] = macd_hist.shift(2)

# CCI
for n in CCI_WINS:
    fe[f'cci_{n}'] = CCIIndicator(high=h, low=l, close=c, window=n).cci()

# Bollinger %b and bandwidth
for n in BB_WINS:
    bb = BollingerBands(close=c, window=n, window_dev=2)
    fe[f'bb_pctb_{n}'] = bb.bollinger_pband()
    fe[f'bb_bw_{n}']   = bb.bollinger_wband()

# Stochastic (%K, %D, slowD) + histogram
for K in STO_KS:
    stoch = StochasticOscillator(high=h, low=l, close=c, window=K, smooth_window=3)
    fast_k = stoch.stoch()
    fast_d = stoch.stoch_signal()
    slow_d = fast_d.rolling(3, min_periods=3).mean()
    fe[f'stoch_fastk_{K}'] = fast_k
    fe[f'stoch_fastd_{K}'] = fast_d
    fe[f'stoch_slowd_{K}'] = slow_d
    fe[f'stoch_hist_{K}']  = fast_k - slow_d

# Price volatility: rolling std of log returns
logret = np.log(c / c.shift(1))
fe[f'volatility_{VOL_WIN}'] = logret.rolling(VOL_WIN, min_periods=VOL_WIN).std()

# Volume features: net volume mean & volume change
fe[f'net_volume_{VOL_MA_VOL}'] = v.rolling(VOL_MA_VOL, min_periods=VOL_MA_VOL).mean()
for n in VOL_ROC_W:
    fe[f'vol_change_{n}'] = v.pct_change(n)

In [7]:
fe

roc_4    roc_12    roc_168  ema_diff_336_672  \
symbol datetime                                                               
ADA    2020-09-04 06:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 07:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 08:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 09:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 10:00:00  7.629973       NaN        NaN               NaN   
...                              ...       ...        ...               ...   
ZRX    2024-02-19 22:00:00  1.552106  1.890990  11.775473          0.003155   
       2024-02-19 23:00:00  0.800000  1.896263  11.538462          0.003234   
       2024-02-20 00:00:00 -0.055157 -0.137779  10.724106          0.003303   
       2024-02-20 01:00:00 -0.766493  0.834492  10.551998          0.003372   
       2024-02-20 02:00:00 -0.409389  1.164403  11.795343          0.003448   

                            ema_ratio_336_672     rsi_24  rsi_24_lag2  \
symbol datetime                                                         
ADA    2020-09-04 06:00:00                NaN        NaN          NaN   
       2020-09-04 07:00:00                NaN        NaN          NaN   
       2020-09-04 08:00:00                NaN        NaN          NaN   
       2020-09-04 09:00:00                NaN        NaN          NaN   
       2020-09-04 10:00:00                NaN        NaN          NaN   
...                                       ...        ...          ...   
ZRX    2024-02-19 22:00:00           0.009490  60.765723    57.787439   
       2024-02-19 23:00:00           0.009725  59.576265    59.922216   
       2024-02-20 00:00:00           0.009931  56.136414    60.765723   
       2024-02-20 01:00:00           0.010136  56.224328    59.576265   
       2024-02-20 02:00:00           0.010360  58.316586    56.136414   

                               rsi_56  rsi_56_lag2    rsi_104  ...  \
symbol datetime                                                ...   
ADA    2020-09-04 06:00:00        NaN          NaN        NaN  ...   
       2020-09-04 07:00:00        NaN          NaN        NaN  ...   
       2020-09-04 08:00:00        NaN          NaN        NaN  ...   
       2020-09-04 09:00:00        NaN          NaN        NaN  ...   
       2020-09-04 10:00:00        NaN          NaN        NaN  ...   
...                               ...          ...        ...  ...   
ZRX    2024-02-19 22:00:00  58.428518    56.987458  56.927229  ...   
       2024-02-19 23:00:00  57.902054    58.016113  56.622820  ...   
       2024-02-20 00:00:00  56.351123    58.428518  55.720276  ...   
       2024-02-20 01:00:00  56.390767    57.902054  55.744018  ...   
       2024-02-20 02:00:00  57.337636    56.351123  56.311683  ...   

                            stoch_slowd_24  stoch_hist_24  stoch_fastk_56  \
symbol datetime                                                             
ADA    2020-09-04 06:00:00             NaN            NaN             NaN   
       2020-09-04 07:00:00             NaN            NaN             NaN   
       2020-09-04 08:00:00             NaN            NaN             NaN   
       2020-09-04 09:00:00             NaN            NaN             NaN   
       2020-09-04 10:00:00             NaN            NaN             NaN   
...                                    ...            ...             ...   
ZRX    2024-02-19 22:00:00       82.041981      11.506406       97.347480   
       2024-02-19 23:00:00       88.323337      -1.226563       94.505495   
       2024-02-20 00:00:00       87.851180     -21.813444       83.229814   
       2024-02-20 01:00:00       82.614679     -15.948013       83.540373   
       2024-02-20 02:00:00       75.661054       6.099953       90.993789   

                            stoch_fastd_56  stoch_slowd_56  stoch_hist_56  \
symbol datetime                                                

In [8]:
fe.describe()

/opt/homebrew/Cellar/jupyterlab/4.4.9/libexec/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/opt/homebrew/Cellar/jupyterlab/4.4.9/libexec/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,roc_4,roc_12,roc_168,ema_diff_336_672,ema_ratio_336_672,rsi_24,rsi_24_lag2,rsi_56,rsi_56_lag2,rsi_104,...,stoch_slowd_24,stoch_hist_24,stoch_fastk_56,stoch_fastd_56,stoch_slowd_56,stoch_hist_56,volatility_80,net_volume_168,vol_change_168,vol_change_336
count,1.485333e+06,1.485325e+06,1.485169e+06,1.484666e+06,1.484666e+06,1.485314e+06,1.485312e+06,1.485282e+06,1.485280e+06,1.485234e+06,...,1.485310e+06,1.485310e+06,1.485282e+06,1.485280e+06,1.485278e+06,1.485278e+06,1.485257e+06,1.485170e+06,1.485169e+06,1.485001e+06
mean,7.646978e+00,2.245936e+01,2.917166e+02,2.675422e-05,-1.413991e-02,4.988681e+01,4.988680e+01,4.978940e+01,4.978939e+01,4.972196e+01,...,5.112515e+01,2.837052e-05,5.041800e+01,5.041798e+01,5.041797e+01,6.243524e-05,1.204820e-02,5.789109e+06,inf,inf
std,4.318938e+03,7.311534e+03,2.538154e+04,3.640503e+02,1.234177e-01,9.034718e+00,9.034720e+00,6.355148e+00,6.355147e+00,5.134597e+00,...,2.543215e+01,1.357320e+01,2.695913e+01,2.637221e+01,2.613712e+01,9.352759e+00,2.399002e-02,3.395866e+07,NaN,NaN
min,-9.999090e+01,-9.999082e+01,-9.999108e+01,-1.116652e+04,-9.808668e-01,1.845119e+00,1.845119e+00,5.061036e+00,5.061036e+00,9.423581e+00,...,2.992986e-04,-7.743281e+01,0.000000e+00,0.000000e+00,1.524299e-04,-7.924422e+01,9.246315e-04,4.813478e+00,-1.000000e+00,-1.000000e+00
25%,-9.825596e-01,-1.774872e+00,-7.106321e+00,-1.799014e-01,-4.153438e-02,4.411780e+01,4.411779e+01,4.600339e+01,4.600339e+01,4.692813e+01,...,2.864182e+01,-7.700804e+00,2.735849e+01,2.747764e+01,2.759259e+01,-4.976142e+00,6.785532e-03,2.071396e+04,-4.849076e-01,-5.046338e-01
50%,9.868547e-03,0.000000e+00,-1.938736e-02,-2.744730e-03,-7.426445e-03,4.987120e+01,4.987118e+01,4.979176e+01,4.979174e+01,4.970943e+01,...,5.101870e+01,2.263769e-01,5.000000e+01,4.990845e+01,4.991901e+01,1.435971e-01,9.620915e-03,1.750316e+05,-2.393247e-02,-2.072740e-02
75%,1.017442e+00,1.823510e+00,7.854985e+00,9.629349e-02,3.127058e-02,5.569234e+01,5.569229e+01,5.370451e+01,5.370449e+01,5.270614e+01,...,7.398160e+01,7.859342e+00,7.372014e+01,7.364136e+01,7.356247e+01,5.125875e+00,1.359884e-02,7.389357e+05,8.861680e-01,9.664933e-01
max,2.690136e+06,2.710919e+06,3.218388e+06,8.994709e+03,9.881140e-01,9.999984e+01,9.999984e+01,9.999958e+01,9.999958e+01,9.999922e+01,...,9.972791e+01,7.540421e+01,1.000000e+02,1.000000e+02,9.972792e+01,8.034365e+01,1.135609e+00,2.178034e+09,inf,inf


In [9]:
# Up/Down/Neutral Labels

# hourly threshold (paper used +/-1% for 4h, +/-1.5% daily). using something in range 0.3%-0.5% since we have hourly data
LABEL_THRESHOLD = 0.004

rp = np.log(df['close'].shift(-1) / df['close'])
y = pd.Series(0, index=df.index)         # neutral
y[rp >  LABEL_THRESHOLD] =  1            # up
y[rp < -LABEL_THRESHOLD] = -1            # down

# Align & drop NaNs (due to warmup windows); last row has NaN label from shift(-1)
Xy = fe.join(y.rename('label')).dropna()
X = Xy.drop(columns='label')
y = Xy['label'].astype(int)

In [10]:
fe

roc_4    roc_12    roc_168  ema_diff_336_672  \
symbol datetime                                                               
ADA    2020-09-04 06:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 07:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 08:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 09:00:00       NaN       NaN        NaN               NaN   
       2020-09-04 10:00:00  7.629973       NaN        NaN               NaN   
...                              ...       ...        ...               ...   
ZRX    2024-02-19 22:00:00  1.552106  1.890990  11.775473          0.003155   
       2024-02-19 23:00:00  0.800000  1.896263  11.538462          0.003234   
       2024-02-20 00:00:00 -0.055157 -0.137779  10.724106          0.003303   
       2024-02-20 01:00:00 -0.766493  0.834492  10.551998          0.003372   
       2024-02-20 02:00:00 -0.409389  1.164403  11.795343          0.003448   

                            ema_ratio_336_672     rsi_24  rsi_24_lag2  \
symbol datetime                                                         
ADA    2020-09-04 06:00:00                NaN        NaN          NaN   
       2020-09-04 07:00:00                NaN        NaN          NaN   
       2020-09-04 08:00:00                NaN        NaN          NaN   
       2020-09-04 09:00:00                NaN        NaN          NaN   
       2020-09-04 10:00:00                NaN        NaN          NaN   
...                                       ...        ...          ...   
ZRX    2024-02-19 22:00:00           0.009490  60.765723    57.787439   
       2024-02-19 23:00:00           0.009725  59.576265    59.922216   
       2024-02-20 00:00:00           0.009931  56.136414    60.765723   
       2024-02-20 01:00:00           0.010136  56.224328    59.576265   
       2024-02-20 02:00:00           0.010360  58.316586    56.136414   

                               rsi_56  rsi_56_lag2    rsi_104  ...  \
symbol datetime                                                ...   
ADA    2020-09-04 06:00:00        NaN          NaN        NaN  ...   
       2020-09-04 07:00:00        NaN          NaN        NaN  ...   
       2020-09-04 08:00:00        NaN          NaN        NaN  ...   
       2020-09-04 09:00:00        NaN          NaN        NaN  ...   
       2020-09-04 10:00:00        NaN          NaN        NaN  ...   
...                               ...          ...        ...  ...   
ZRX    2024-02-19 22:00:00  58.428518    56.987458  56.927229  ...   
       2024-02-19 23:00:00  57.902054    58.016113  56.622820  ...   
       2024-02-20 00:00:00  56.351123    58.428518  55.720276  ...   
       2024-02-20 01:00:00  56.390767    57.902054  55.744018  ...   
       2024-02-20 02:00:00  57.337636    56.351123  56.311683  ...   

                            stoch_slowd_24  stoch_hist_24  stoch_fastk_56  \
symbol datetime                                                             
ADA    2020-09-04 06:00:00             NaN            NaN             NaN   
       2020-09-04 07:00:00             NaN            NaN             NaN   
       2020-09-04 08:00:00             NaN            NaN             NaN   
       2020-09-04 09:00:00             NaN            NaN             NaN   
       2020-09-04 10:00:00             NaN            NaN             NaN   
...                                    ...            ...             ...   
ZRX    2024-02-19 22:00:00       82.041981      11.506406       97.347480   
       2024-02-19 23:00:00       88.323337      -1.226563       94.505495   
       2024-02-20 00:00:00       87.851180     -21.813444       83.229814   
       2024-02-20 01:00:00       82.614679     -15.948013       83.540373   
       2024-02-20 02:00:00       75.661054       6.099953       90.993789   

                            stoch_fastd_56  stoch_slowd_56  stoch_hist_56  \
symbol datetime                                                

In [11]:
fe.count()

roc_4                1485333
roc_12               1485325
roc_168              1485169
ema_diff_336_672     1484666
ema_ratio_336_672    1484666
rsi_24               1485314
rsi_24_lag2          1485312
rsi_56               1485282
rsi_56_lag2          1485280
rsi_104              1485234
rsi_104_lag2         1485232
macd_hist            1485304
macd_hist_lag2       1485302
cci_40               1485298
cci_80               1485258
bb_pctb_40           1263900
bb_bw_40             1485298
bb_pctb_80           1485258
bb_bw_80             1485258
stoch_fastk_24       1485314
stoch_fastd_24       1485312
stoch_slowd_24       1485310
stoch_hist_24        1485310
stoch_fastk_56       1485282
stoch_fastd_56       1485280
stoch_slowd_56       1485278
stoch_hist_56        1485278
volatility_80        1485257
net_volume_168       1485170
vol_change_168       1485169
vol_change_336       1485001
dtype: int64

In [12]:
# Normalize data if not already normalized

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import TimeSeriesSplit

num_cols = X.select_dtypes(np.number).columns.tolist()

# already bound 0-100 or 0-1
bounded_cols   = [c for c in num_cols if c.startswith(('rsi_', 'stoch_', 'bb_pctb_'))]
# log1p then get z-score
loggy_cols     = [c for c in num_cols if c.startswith(('net_volume_', 'vol_change_', 'bb_bw_', 'volatility_', 'ema_ratio_'))]
# z-score
the_rest       = sorted(list(set(num_cols) - set(bounded_cols) - set(loggy_cols)))

def signed_log1p(df):
    Z = df.copy()
    for c in Z.columns:
        x = Z[c].values
        Z[c] = np.sign(x) * np.log1p(np.abs(x))
    return Z

pre = ColumnTransformer(
    transformers=[
        ("bounded_passthrough", "passthrough", bounded_cols),
        ("log_then_standardize", Pipeline([
            ("log",       FunctionTransformer(signed_log1p, validate=False)),
            ("standard",  StandardScaler())
        ]), loggy_cols),
        ("standardize_rest", StandardScaler(), the_rest)
    ],
    remainder="drop"
)


In [14]:
# Train-Test Split and Model Training

used_coins = []
skipped_coins = []

for sym, g in X.groupby('symbol', sort=False):

    n = len(g)
    max_splits = (n - 800) // 2160   # ensure ~800 training rows for lookback window
    actual_splits = min(8, max_splits)

    # use coins that have a reasonable amount of data
    if actual_splits < 4:
        skipped_coins.append(sym)
        continue

    used_coins.append(sym)
    
    tscv = TimeSeriesSplit(n_splits=actual_splits, test_size=2160)
    
    for train_idx, test_idx in tscv.split(g):
        X_train, X_test = g.iloc[train_idx], g.iloc[test_idx]
        y_train, y_test = y.loc[X_train.index], y.loc[X_test.index]
        
        # fit scaler on X_train only
        # fit model → evaluate on X_test

In [22]:
print("Used:", used_coins)
print("\n")
print("Skipped:", skipped_coins)

Used: ['ADA', 'ALGO', 'ATOM', 'BAL', 'BAND', 'BAT', 'BCH', 'BNB', 'BTC', 'COMP', 'CRV', 'DASH', 'DOT', 'EGLD', 'EOS', 'ETC', 'ETH', 'ICX', 'IOTA', 'KAVA', 'KNC', 'LINK', 'LTC', 'MKR', 'NEO', 'OMG', 'ONT', 'QTUM', 'RLC', 'RUNE', 'SNX', 'SOL', 'SUSHI', 'SXP', 'THETA', 'TRB', 'WAVES', 'XLM', 'XMR', 'XRP', 'XTZ', 'YFI', 'ZEC', 'ZRX']


Skipped: ['DOGE', 'IOST', 'TRX', 'VET', 'ZIL']
